In [1]:
import nltk
import numpy as np
import pandas as pd

from collections import Counter, OrderedDict
from nltk.tokenize import word_tokenize, TreebankWordTokenizer
from nltk.corpus import stopwords

import copy
import math

stops = stopwords.words('english')

In [2]:
txt1 = """The bustling city of New York is a melting pot of cultures and a hub for business and innovation. With its iconic skyline, Central Park, and the Statue of Liberty, New York City has a unique charm that draws millions of tourists every year. The city that never sleeps is known for its diverse neighborhoods, such as Chinatown, Little Italy, and Harlem, each offering a distinct cultural experience. The subway system, yellow taxis, and the famous Times Square are symbols of the city's energy and pace of life. New York is home to Wall Street, the epicenter of global finance, and Silicon Alley, a burgeoning tech hub. Whether you're exploring art at the Metropolitan Museum, enjoying a Broadway show, or savoring a classic New York-style pizza, the Big Apple has something for everyone."""
txt2 = """The Great Barrier Reef is a natural wonder of the world, located off the coast of Queensland, Australia. It is the largest coral reef system on the planet, spanning over 2,300 kilometers and comprising thousands of individual reefs and islands. The reef is a haven for marine biodiversity, housing a stunning array of coral species, fish, turtles, sharks, and other marine life. Snorkeling and diving in the crystal-clear waters of the Great Barrier Reef offer a chance to witness this underwater paradise up close. Tourists from around the globe flock to explore its vibrant coral formations, including the renowned Heart Reef. This ecological wonder, however, faces challenges from climate change and pollution, making conservation efforts crucial to protect its delicate ecosystem."""

In [3]:
def token_pro(text):
    tokens = word_tokenize(text)
    tokens = [word.lower() for word in tokens] # everything lowercase
    tokens = [word.strip() for word in tokens] # strips whitespaces
    tokens = [word for word in tokens if word.isalnum()] # only alpha numerical values
    tokens = [word for word in tokens if not word in stops] # removes stopwords
    return tokens 

def genBow(tokens):
    bow = Counter(tokens)
    return bow

def printBow(bow, queryString):
    query_frequency = Counter(queryString)
    tf_vec = []
    for word in bow:
        if word in queryString:
            tf = query_frequency[word] #/len(bow)
        else:
            tf = 0
        tf_vec.append(tf)
    return tf_vec

In [4]:
corpus = [txt1, txt2]
corpus_tokens = token_pro((' ').join(corpus))
corpus_tokens

['bustling',
 'city',
 'new',
 'york',
 'melting',
 'pot',
 'cultures',
 'hub',
 'business',
 'innovation',
 'iconic',
 'skyline',
 'central',
 'park',
 'statue',
 'liberty',
 'new',
 'york',
 'city',
 'unique',
 'charm',
 'draws',
 'millions',
 'tourists',
 'every',
 'year',
 'city',
 'never',
 'sleeps',
 'known',
 'diverse',
 'neighborhoods',
 'chinatown',
 'little',
 'italy',
 'harlem',
 'offering',
 'distinct',
 'cultural',
 'experience',
 'subway',
 'system',
 'yellow',
 'taxis',
 'famous',
 'times',
 'square',
 'symbols',
 'city',
 'energy',
 'pace',
 'life',
 'new',
 'york',
 'home',
 'wall',
 'street',
 'epicenter',
 'global',
 'finance',
 'silicon',
 'alley',
 'burgeoning',
 'tech',
 'hub',
 'whether',
 'exploring',
 'art',
 'metropolitan',
 'museum',
 'enjoying',
 'broadway',
 'show',
 'savoring',
 'classic',
 'new',
 'pizza',
 'big',
 'apple',
 'something',
 'everyone',
 'great',
 'barrier',
 'reef',
 'natural',
 'wonder',
 'world',
 'located',
 'coast',
 'queensland',
 'aus